## Load Libraries

In [20]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
from PIL import Image
import io
import os

## 1. PPT에서 텍스트/표/이미지 분리 추출

In [21]:
def extract_pptx_elements(pptx_path, output_dir='img_output'):
    prs = Presentation(pptx_path)
    os.makedirs(output_dir, exist_ok=True)

    slides_data = []

    for i, slide in enumerate(prs.slides):
        slide_info = {
            'slide_num': i + 1,
            'texts': [],
            'tables': [],
            'images': []
        }

        for shape in slide.shapes:
            if shape.shape_type == MSO_SHAPE_TYPE.TEXT_BOX or shape.has_text_frame:
                text = shape.text.strip()
                if text:
                    slide_info['texts'].append(text)

            elif shape.shape_type == MSO_SHAPE_TYPE.TABLE:
                table_data = []
                for row in shape.table.rows:
                    row_data = [cell.text.strip() for cell in row.cells]
                    table_data.append(row_data)
                slide_info['tables'].append(table_data)

            elif shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                image = shape.image
                image_bytes = image.blob
                image_filename = f"slide_{i+1}_img_{len(slide_info['images']) + 1}.{image.ext}"
                with open(os.path.join(output_dir, image_filename), 'wb') as f:
                    f.write(image_bytes)
                slide_info['images'].append(image_filename)

        slides_data.append(slide_info)

    return slides_data

In [22]:
pptx_path = 'data/sample_data.pptx'
slides_data = extract_pptx_elements(pptx_path)

## 2. 슬라이드 단위로 요약 입력 정제

In [23]:
def generate_slide_summaries(slides_data, output_dir='img_output'):
    summaries = []

    for slide in slides_data:
        summary_parts = []
        slide_num = slide['slide_num']
        summary_parts.append(f"[Slide {slide_num}]")

        # 1. 텍스트 요약
        if slide['texts']:
            summary_parts.append("텍스트 요약:")
            summary_parts.extend(slide['texts'])

        # 2. 표 요약
        for table in slide['tables']:
            if table:
                header = table[0]
                rows = table[1:]
                table_summary = "표 내용 요약: "
                table_summary += ", ".join([f"{row[0]}: {row[1]}" for row in rows if len(row) >= 2])
                summary_parts.append(table_summary)

        # 3. 이미지 언급
        if slide['images']:
            img_files = [f"{output_dir}/{img}" for img in slide['images']]
            summary_parts.append(f"이미지 파일 포함: {', '.join(img_files)}")

        # Join all parts
        slide_summary = "\n".join(summary_parts)
        summaries.append(slide_summary)

    return summaries

In [24]:
summaries = generate_slide_summaries(slides_data)

## 3. 로컬 LLM 기반 질의응답 RAG

1. PPT 슬라이드 요약 텍스트
2. OllamaEmbeddings (llama3 기반)
3. FAISS 벡터 DB
4. Ollama 로컬 LLM (llama3:8b)
5. 사용자 질문 → 답변 생성

In [25]:
# ollama run llama3

#### Langchain에서 Ollama 모델 활용

In [26]:
# from langchain_ollama import OllamaLLM
# llm = OllamaLLM(model="llama3:8b")

### 1) 임베딩 + 벡터 저장 (로컬 LLM 기반)

In [27]:
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document

#### 임베딩 생성

In [28]:
embedding_model = OllamaEmbeddings(model="exaone3.5:7.8b")

#### 슬라이드 요약 내용을 문서화

In [29]:
docs = [Document(page_content=summary) for summary in summaries]

In [30]:
docs

[Document(metadata={}, page_content='[Slide 1]\n텍스트 요약:\nPC방 창업 사업계획서\nIT/인터넷 관련 사업계획서\n경영기획팀\n2025. 04. 05\n이미지 파일 포함: img_output/slide_1_img_1.png'),
 Document(metadata={}, page_content='[Slide 2]\n텍스트 요약:\nCONTENTS'),
 Document(metadata={}, page_content='[Slide 3]\n텍스트 요약:\n1. 회사현황\n1) 회사개요\n회사의 현황에 따른 개요는 아래와 같음\n세부 사항은 아래에 상세히 설명되어 있음\n표 내용 요약: 대표자명: 김효섭, 자택전화번호: 02) 123-4567, 사업장소재지: 서울시 성동구 아차산로 49 서울숲 코오롱 디지털타워, 업 태: 서비스, 창업(예정)일자: 20 . ., 특기사항: \n이미지 파일 포함: img_output/slide_3_img_1.jpg'),
 Document(metadata={}, page_content='[Slide 4]\n텍스트 요약:\n1. 회사현황\n2) 창업자의 인적사항\n창업자 인적사항은 아래와 같음\n세부 사항은 아래에 상세히 설명되어 있음\n표 내용 요약: 주 소: 서울시 광진구 자양동, 학 력: 기 간, : ～, : ～, : ~, 경 력: 근무기간, : ～, : ～'),
 Document(metadata={}, page_content='[Slide 5]\n텍스트 요약:\n2. 사업계획\n1) 사업의 등기 및 개요\n가) 사업의 동기\n본인은 재직 중이던 회사의 부도로 인하여 20  년   월 회사를 사직하였다.\n회사를 사직하고 취업을 하려해도 나이 때문에 여의치 않아 생계유지 차원에서 인터넷멀티게임장(PC방) 창업을 계획하게 되었다. \n본 사업을 아이템으로 정한 동기는 현재 하고 있는 친구의 권유로 관심을 가지고 조사하여 본 결과 본인의 적성, 수행능력, 자금력, 투자수익 등이 적정하다고 판단되

#### FAISS 벡터 DB 생성/저장/로드

In [31]:
# FAISS DB 생성
db = FAISS.from_documents(docs, embedding_model)

In [32]:
# FAISS DB 저장
db.save_local("faiss_index/slide_vector_db")

In [33]:
# FAISS DB 로드 (생성 후 사용)
# db = FAISS.load_local("faiss_index/slide_vector_db", embeddings=embedding_model)

### 2) 로컬 LLM 질의응답

#### LCEL 방식

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

# Ollama 로컬 실행 모델 연결
llm = OllamaLLM(model="exaone3.5:7.8b")

# 프롬프트 정의
prompt = PromptTemplate(
    input_variables=["context", "input"],
    template="""
    당신은 친절한 AI 어시스턴트입니다. 다음의 문맥 정보를 참고하여 질문에 정확하게 답하세요.

    문맥:
    {context}

    질문:
    {input}

    답변:
    """
)

# 문서 결합 방식 정의
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# 최종 QA 체인 생성
qa_chain = create_retrieval_chain(
    retriever=db.as_retriever(),  # 벡터스토어에서 as_retriever 사용
    combine_docs_chain=combine_docs_chain
)

# 실행
response = qa_chain.invoke({"input": "추정손익계산서의 감가상각비 금액은 얼마인가요?"})
print(response["answer"])

추정손익계산서의 감가상각비 금액은 **23,600,000**입니다. 이 금액은 [Slide 11]의 고정비 계획 섹션에서 제공된 표 내용 요약에 명시되어 있습니다.


### 기존 방식 (ollama3:8b)

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

# Ollama 로컬 실행 모델 연결
llm = OllamaLLM(model="exaone3.5:7.8b")

# 질의응답 체인 구성 (from_chain_type > from_llm)
qa = RetrievalQA.from_llm(
    llm=llm,
    retriever=db.as_retriever()
)

In [17]:
# 실행
question = "추정손익계산서의 감가상각비 금액은 얼마인가요?"
answer = qa.invoke(question)
print(answer)

{'query': '추정손익계산서의 감가상각비 금액은 얼마인가요?', 'result': 'I can help you with that! According to Slide 11, the depreciation expense (감가상각비) is 23,600,000.'}


In [18]:
# 실행
question = "창업자의 성명은 무엇인가요?"
answer = qa.invoke(question)
print(answer)

{'query': '창업자의 성명은 무엇인가요?', 'result': "I don't know. The provided context does not mention the entrepreneur's name."}


In [19]:
# 실행
question = "이 보고서의 전체 내용을 간략히 요약해 줘"
answer = qa.invoke(question)
print(answer)

{'query': '이 보고서의 전체 내용을 간략히 요약해 줘', 'result': "I don't know the answer. The provided context appears to be a series of slides with text summaries in Korean, but I'm unable to summarize the entire report without understanding the content. Can you provide more context or translate the text for me?"}
